In [1]:
from sqlalchemy import Boolean, Column, Float, ForeignKey, Integer, MetaData, String, Table, create_engine, select
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import Session, relationship

In [2]:
engine = create_engine("sqlite+pysqlite:///./relations.sqlite", echo=False, future=True)

In [3]:
metadata = MetaData()

books_table = Table(
    "books",
    metadata,
    Column("id", Integer, primary_key=True),
    Column("title", String),
    Column("isbn", String, unique=True, nullable=False),
)

copies_table = Table(
    "copies",
    metadata,
    Column("id", Integer, primary_key=True),
    Column("status", String),  # (bad, good, excellent)
    Column("book_id", ForeignKey("books.id"), nullable=False),
)

authors_table = Table(
    "authors",
    metadata,
    Column("id", Integer, primary_key=True),
    Column("firstname", String),
    Column("lastname", String),
)

books_authors_table = Table(
    "books_authors",
    metadata,
    Column("book_id", Integer, ForeignKey("books.id")),
    Column("author_id", Integer, ForeignKey("authors.id")),
)

metadata.create_all(bind=engine)

In [4]:
Base = declarative_base()


class Book(Base):
    __table__ = books_table

    copies = relationship("Copy", back_populates="book", cascade="all, delete")
    authors = relationship("Author", back_populates="books", secondary=books_authors_table)

    def __repr__(self):
        return f"<Book {self.id}>"


class Copy(Base):
    __table__ = copies_table

    book = relationship("Book", back_populates="copies")

    def __repr__(self):
        return f"<Copy {self.id}>"

    @property
    def title(self):
        return self.book.title


class Author(Base):
    __table__ = authors_table

    books = relationship("Book", back_populates="authors", secondary=books_authors_table)

    def __repr__(self):
        return f"<Author {self.id}>"

In [5]:
session = Session(bind=engine)

In [6]:
book1 = Book(title="Book 1", isbn="111")
session.add(book1)
session.commit()

In [7]:
book2 = Book(title="Book 2", isbn="222")
session.add(book2)
session.commit()

In [8]:
book1_copy1 = Copy(status="bad", book_id=1)
session.add(book1_copy1)
book1_copy2 = Copy(status="good", book_id=1)
session.add(book1_copy2)
session.commit()

In [9]:
book2_copy1 = Copy(status="excellent", book_id=2)
session.add(book2_copy1)
book2_copy2 = Copy(status="excellent", book=book2)
session.add(book2_copy2)
session.commit()

In [10]:
for c in book1.copies:
    print(c.id, c.status)

1 bad
2 good


In [11]:
author1 = Author(firstname="Sébastien", lastname="Verbois")
session.add(author1)
author2 = Author(firstname="Alain", lastname="Meurant")
session.add(author2)
session.commit()

In [12]:
book1 = session.get(Book, 1)
book1.authors.append(author1)
session.commit()

In [13]:
statement = select(Book).where(Book.isbn == "222")
book2 = session.execute(statement).scalars().one()
book2.authors.append(author1)
book2.authors.append(author2)
session.commit()

In [14]:
statement = select(Book)  # .where(books_table.c.isbn == "222")
session.execute(statement).all()

[(<Book 1>,), (<Book 2>,)]

In [15]:
book2 = session.get(Book, 2)
session.delete(book2)
session.commit()

In [16]:
book3 = Book(title="Book 3", isbn="333")
author3 = Author(firstname="Jean", lastname="Dupont")
book3.authors.append(author3)
session.add(book3)
session.commit()

In [17]:
statement = select(Book, Author).join(Book.authors)
session.execute(statement).all()

[(<Book 1>, <Author 1>), (<Book 2>, <Author 3>)]